### Install necessary HuggingFace libraries

In [2]:
!pip install datasets

  Using cached datasets-2.3.2-py3-none-any.whl (362 kB)
     |████████████████████████████████| 22.4 MB 5.0 MB/s eta 0:00:01
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached multiprocess-0.70.13-py39-none-any.whl (132 kB)
  Using cached huggingface_hub-0.8.1-py3-none-any.whl (101 kB)


In [3]:
!pip install transformers

  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
     |████████████████████████████████| 3.6 MB 5.8 MB/s eta 0:00:01


In [4]:
!pip install torch

In [33]:
!pip install tensorflow

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 228.5 MB 26.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 7.3 MB/s  eta 0:00:01
     |████████████████████████████████| 123 kB 20.4 MB/s eta 0:00:01
     |████████████████████████████████| 438 kB 23.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 27.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 25.2 MB/s eta 0:00:01
     |████████████████████████████████| 13.2 MB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 23.5 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 41.0 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 13.4 MB

### Import data

In [22]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100] # data example

Reusing dataset yelp_review_full (/Users/galogonzalvo/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

### Tokenize dataset

**Padding**: Add a special *padding token* to ensure shorter sequences will have the same length as either the longest sequence in a batch or the maximum length accepted by the model, depending on the selected option.

**Truncating**: Works in the other direction, truncating long sequences.

In [23]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") #Load pre-trained tokenizer model

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length",truncation=True) #Apply padding and truncation

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets["train"][100]

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /Users/galogonzalvo/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/re

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

### Select subset to speed-up training

In [7]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### Load your pre-trained model

In this case, we use a model for **Sequence Classification**

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

### Train your model

1) Create a ```TrainingArguments``` class which contains all the hyperparameters.

2) To compute and report metrics during training, load the accuracy function using ```load_metric``` and apply them in the different training steps. To do so, one first needs to convert the predictions to [logits](https://en.wikipedia.org/wiki/Logit).

3) Create a ```Trainer```object and fine-tune the model by calling ```Trainer.train()```.

In [10]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
metric

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred 
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels) #predictions are the predicted labels, references are the ground truth labels

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch") #Perform evaluation after each epoch

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

### Using your own pre-trained model

In [17]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")


loading configuration file https://huggingface.co/Davlan/bert-base-multilingual-cased-ner-hrl/resolve/main/config.json from cache at /Users/galogonzalvo/.cache/huggingface/transformers/f9d83543e7b4d512a49f501e3cc81d1f6354be3e16badffd33932d65bc95934f.c6004f85a5480d0dfa44756af37f81963685213d32fbe40b0fb42c1e6c9999b1
Model config BertConfig {
  "_name_or_path": "Davlan/bert-base-multilingual-cased-ner-hrl",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-DATE",
    "2": "I-DATE",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DATE": 1,
    "B-LOC": 7,
    "B-ORG": 5,
    "B-PER": 3,
    "I-

In [35]:
dataset = load_dataset('polyglot_ner', 'es')
dataset['train'][100]

Reusing dataset polyglot_ner (/Users/galogonzalvo/.cache/huggingface/datasets/polyglot_ner/es/1.0.0/bb2e45c90cd345c87dfd757c8e2b808b78b0094543b511ac49bc0129699609c1)


  0%|          | 0/1 [00:00<?, ?it/s]

{'id': '100',
 'lang': 'es',
 'words': ['Dado',
  'que',
  'los',
  'discos',
  'DVD',
  '+',
  'R',
  '/',
  'RW',
  'no',
  'forman',
  'parte',
  'de',
  'los',
  'estándares',
  'oficiales',
  ',',
  'no',
  'muestran',
  'el',
  'logotipo',
  '«',
  'DVD',
  '».'],
 'ner': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']}

In [37]:
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at /Users/galogonzalvo/.cache/huggingface/datasets/polyglot_ner/es/1.0.0/bb2e45c90cd345c87dfd757c8e2b808b78b0094543b511ac49bc0129699609c1/cache-d13e9229d29a6f46.arrow


In [29]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",label_names=['ner'])
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [30]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: text. If text are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


ValueError: Expected input batch_size (4096) to match target batch_size (8).

### Using Keras

In [31]:
from transformers import DefaultDataCollator

# The DefaultDataCollator assembles tensors into a batch for model to train on
data_collator = DefaultDataCollator(return_tensors="tf")

### Convert dataset to TensorFlow format

In [38]:
tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns=["id", "lang", "words"],
    label_cols=["ner"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8
)

ImportError: Called a Tensorflow-specific function but Tensorflow is not installed.

In [ ]:
import tensorflow as tf